# Project predict power in HackerEarth

In [13]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from Clean import missing as ms

In [2]:
train = pd.read_csv("Data/power-data/dataset/train.csv", delimiter=',')
test = pd.read_csv("Data/power-data/dataset/test.csv")
#train = pd.read_csv("Data/power-data/dataset/tran_fix.csv", delimiter=';')

In [3]:
print(train.shape)
print(test.shape)

(28200, 22)
(12086, 21)


# Data Processing

In [4]:
c_train = train.copy(deep=True)
c_test = test.copy(deep=True)

#c_train['train'] = 1
#c_test['train'] = 0
#df = pd.concat([c_train, c_test], axis=0, sort=False)

check missing data

In [5]:
def missing_values(df):
    mis_val = df.isnull().sum() #tổng các giá trị còn thiếu
    mis_val_per = 100 * df.isnull().sum()/len(df) #Phần trăm các giá trị còn thiếu
    mis_val_table = pd.concat([mis_val, mis_val_per], axis=1) #Tạo thành bảng để thể hiện 2 giá trị trên
    mis_val_table_rename = mis_val_table.rename(
    columns = {0: "Missing", 1 : "Percent %"})
    #Sắp xếp bảng theo tỉ lệ phần trăm giảm dần
    mis_val_table_rename = mis_val_table_rename[mis_val_table_rename.iloc[:, 1] != 0].sort_values("Percent %", ascending=False).round(1)
    print("Có tất cả " + str(df.shape[1]) + " cột.\n"
          "Trog đó có " + str(mis_val_table_rename.shape[0]) + " cột bị thiếu data")
    return mis_val_table_rename

## Train

In [6]:
missing_values(c_train)

Có tất cả 22 cột.
Trog đó có 18 cột bị thiếu data


,Missing,Percent %
wind_direction(°),5103,18.1
blade_length(m),5093,18.1
atmospheric_temperature(°C),3450,12.2
atmospheric_pressure(Pascal),2707,9.6
windmill_body_temperature(°C),2363,8.4
turbine_status,1759,6.2
rotor_torque(N-m),572,2.0
windmill_height(m),543,1.9
cloud_level,276,1.0
wind_speed(m/s),273,1.0


In [7]:
# trong nhan label co thieu missing data nen loai bo no di
c_train =  c_train.dropna(subset=['windmill_generated_power(kW/h)'])

In [8]:
# Fill numeric data by values: 'mean'
c_train = c_train.fillna(c_train.mean())

In [9]:
missing_values(c_train)

Có tất cả 22 cột.
Trog đó có 2 cột bị thiếu data


,Missing,Percent %
turbine_status,1747,6.2
cloud_level,272,1.0


In [10]:
numeric_data = c_train.select_dtypes(exclude='object')
category_data = c_train.select_dtypes(include='object')

In [21]:
missing_values(numeric_data)

Có tất cả 18 cột.
Trog đó có 0 cột bị thiếu data


,Missing,Percent %


In [12]:
# check missing in category data
missing_values(category_data)

Có tất cả 4 cột.
Trog đó có 2 cột bị thiếu data


,Missing,Percent %
turbine_status,1747,6.2
cloud_level,272,1.0


In [16]:
category_data = ms.impute_NA_with_avg(data=category_data, strategy='mode', NA_col=["turbine_status", "cloud_level"])

In [17]:
category_data.head()

,tracking_id,datetime,turbine_status,cloud_level,turbine_status_impute_mode,cloud_level_impute_mode
0,WM_33725,2019-08-04 14:33:20,BA,Medium,BA,Medium
1,WM_698,2018-11-05 10:13:20,A2,Medium,A2,Medium
2,WM_39146,2019-09-14 14:03:20,ABC,Medium,ABC,Medium
3,WM_6757,2018-12-25 15:33:20,ABC,NaN,ABC,Low
4,WM_21521,2019-05-04 03:13:20,AAA,Medium,AAA,Medium


In [18]:
category_data = category_data.dropna(axis=1)
category_data.head()

,tracking_id,datetime,turbine_status_impute_mode,cloud_level_impute_mode
0,WM_33725,2019-08-04 14:33:20,BA,Medium
1,WM_698,2018-11-05 10:13:20,A2,Medium
2,WM_39146,2019-09-14 14:03:20,ABC,Medium
3,WM_6757,2018-12-25 15:33:20,ABC,Low
4,WM_21521,2019-05-04 03:13:20,AAA,Medium


In [22]:
train_concat = pd.concat([category_data, numeric_data], axis=1, sort=False)
train_concat.head()

,tracking_id,datetime,turbine_status_impute_mode,cloud_level_impute_mode,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),...,atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h)
0,WM_33725,2019-08-04 14:33:20,BA,Medium,94.820023,-99.000000,41.723019,-0.903423,82.410573,42.523015,...,103402.961872,26.897875,20.767246,239.836388,2730.310605,42.084666,2.217542,0.314065,24.281689,6.766521
1,WM_698,2018-11-05 10:13:20,A2,Medium,241.832734,27.764785,-99.000000,-99.000000,44.104919,46.258870,...,17030.904078,39.801469,20.767246,337.944723,1780.207200,107.888643,4.210346,0.448494,27.262139,5.966275
2,WM_39146,2019-09-14 14:03:20,ABC,Medium,95.484724,0.371184,41.855473,12.652763,42.322098,42.878552,...,16125.927107,36.116065,45.033197,227.850294,1666.049900,-42.931459,2.719475,0.302321,27.366127,2.874342
3,WM_6757,2018-12-25 15:33:20,ABC,Low,238.819424,-99.000000,45.443914,15.115323,44.759643,47.282101,...,18689.732336,46.020045,44.827154,492.081520,1964.502895,42.744596,4.857385,0.367140,24.287767,14.851089
4,WM_21521,2019-05-04 03:13:20,AAA,Medium,10.722890,0.371184,41.981183,1.715696,-17.616459,43.469852,...,114468.169007,34.572941,-99.000000,259.274601,1177.516152,13.387289,2.248762,0.453374,27.971650,3.519074


In [23]:
# check end missing
missing_values(train_concat)

Có tất cả 22 cột.
Trog đó có 0 cột bị thiếu data


,Missing,Percent %


## Test

In [24]:
missing_values(c_test)

Có tất cả 21 cột.
Trog đó có 17 cột bị thiếu data


,Missing,Percent %
wind_direction(°),2160,17.9
blade_length(m),2114,17.5
atmospheric_temperature(°C),1427,11.8
atmospheric_pressure(Pascal),1151,9.5
windmill_body_temperature(°C),926,7.7
turbine_status,797,6.6
rotor_torque(N-m),281,2.3
windmill_height(m),255,2.1
wind_speed(m/s),126,1.0
cloud_level,125,1.0


In [25]:
c_test = c_test.fillna(c_test.mean())
missing_values(c_test)

Có tất cả 21 cột.
Trog đó có 2 cột bị thiếu data


,Missing,Percent %
turbine_status,797,6.6
cloud_level,125,1.0


In [26]:
numeric_data = c_test.select_dtypes(exclude='object')
category_data = c_test.select_dtypes(include='object')

In [27]:
missing_values(category_data)

Có tất cả 4 cột.
Trog đó có 2 cột bị thiếu data


,Missing,Percent %
turbine_status,797,6.6
cloud_level,125,1.0


In [28]:
category_data = ms.impute_NA_with_avg(data=category_data, strategy='mode', NA_col=["turbine_status", "cloud_level"])

In [29]:
category_data = category_data.dropna(axis=1)
category_data.head()

,tracking_id,datetime,turbine_status_impute_mode,cloud_level_impute_mode
0,WM_19817,2019-04-17 08:53:20,BA,Medium
1,WM_18723,2019-03-30 07:43:20,A,Medium
2,WM_34552,2019-08-10 11:33:20,B2,Medium
3,WM_28570,2019-06-26 03:53:20,BBB,Low
4,WM_36934,2019-08-27 16:43:20,A,Low


In [30]:
test_concat = pd.concat([category_data, numeric_data], axis=1, sort=False)
test_concat.head()

,tracking_id,datetime,turbine_status_impute_mode,cloud_level_impute_mode,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),...,generator_temperature(°C),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m)
0,WM_19817,2019-04-17 08:53:20,BA,Medium,94.324266,17.641186,89.714193,51.146788,40.460560,39.594734,...,66.830037,16681.044117,24.004812,43.756693,445.976992,1664.222023,21.912243,3.185837,0.403965,25.572431
1,WM_18723,2019-03-30 07:43:20,A,Medium,10.088870,13.978119,43.272846,46.516394,40.027788,41.176860,...,37.284163,54283.324683,29.431813,42.728174,499.595287,1165.111992,-35.050093,3.016603,0.444755,24.371823
2,WM_34552,2019-08-10 11:33:20,B2,Medium,347.152090,31.423035,41.076640,26.931602,43.109122,43.439556,...,70.010762,214812.836175,29.924235,43.256122,245.432231,1667.720491,27.195302,2.611941,0.387368,27.654677
3,WM_28570,2019-06-26 03:53:20,BBB,Low,24.471997,-99.000000,14.375078,66.513953,13.741253,15.577472,...,41.445258,54283.324683,23.886434,13.501595,308.376140,1329.744740,15.245757,2.866805,0.450478,24.189426
4,WM_36934,2019-08-27 16:43:20,A,Low,96.997026,33.281836,41.405192,1.843112,121.572907,43.934587,...,68.007787,16833.546517,35.906889,-99.000000,442.425744,691.408996,34.257024,3.549672,0.368355,4.885440


## Data

In [40]:
train_concat['train'] = 1
test_concat['train'] = 0
df = pd.concat([train_concat, test_concat], axis=0, sort=False)

In [41]:
df.head()

,tracking_id,datetime,turbine_status_impute_mode,cloud_level_impute_mode,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),...,area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h),train
0,WM_33725,2019-08-04 14:33:20,BA,Medium,94.820023,-99.000000,41.723019,-0.903423,82.410573,42.523015,...,26.897875,20.767246,239.836388,2730.310605,42.084666,2.217542,0.314065,24.281689,6.766521,1
1,WM_698,2018-11-05 10:13:20,A2,Medium,241.832734,27.764785,-99.000000,-99.000000,44.104919,46.258870,...,39.801469,20.767246,337.944723,1780.207200,107.888643,4.210346,0.448494,27.262139,5.966275,1
2,WM_39146,2019-09-14 14:03:20,ABC,Medium,95.484724,0.371184,41.855473,12.652763,42.322098,42.878552,...,36.116065,45.033197,227.850294,1666.049900,-42.931459,2.719475,0.302321,27.366127,2.874342,1
3,WM_6757,2018-12-25 15:33:20,ABC,Low,238.819424,-99.000000,45.443914,15.115323,44.759643,47.282101,...,46.020045,44.827154,492.081520,1964.502895,42.744596,4.857385,0.367140,24.287767,14.851089,1
4,WM_21521,2019-05-04 03:13:20,AAA,Medium,10.722890,0.371184,41.981183,1.715696,-17.616459,43.469852,...,34.572941,-99.000000,259.274601,1177.516152,13.387289,2.248762,0.453374,27.971650,3.519074,1


In [42]:
missing_values(df)

Có tất cả 23 cột.
Trog đó có 1 cột bị thiếu data


,Missing,Percent %
windmill_generated_power(kW/h),12086,30.2


----> Nice

# Encode Data

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40079 entries, 0 to 12085
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   tracking_id                     40079 non-null  object 
 1   datetime                        40079 non-null  object 
 2   turbine_status_impute_mode      40079 non-null  object 
 3   cloud_level_impute_mode         40079 non-null  object 
 4   wind_speed(m/s)                 40079 non-null  float64
 5   atmospheric_temperature(°C)     40079 non-null  float64
 6   shaft_temperature(°C)           40079 non-null  float64
 7   blades_angle(°)                 40079 non-null  float64
 8   gearbox_temperature(°C)         40079 non-null  float64
 9   engine_temperature(°C)          40079 non-null  float64
 10  motor_torque(N-m)               40079 non-null  float64
 11  generator_temperature(°C)       40079 non-null  float64
 12  atmospheric_pressure(Pascal)    

for 2 category feature: turbine_status_impute_mode & cloud_level_impute_mode

In [43]:
df.cloud_level_impute_mode.value_counts()

Low              20153
Medium           19491
Extremely Low      435
Name: cloud_level_impute_mode, dtype: int64

In [44]:
df['cloud_level_impute_mode'] = df.cloud_level_impute_mode.replace('Extremely Low', 'Medium')
df.cloud_level_impute_mode.value_counts()

Low       20153
Medium    19926
Name: cloud_level_impute_mode, dtype: int64

In [47]:
df.turbine_status_impute_mode.value_counts()

BB     5345
B2     2777
A      2766
BCB    2731
AAA    2698
D      2693
AB     2659
BA     2644
ABC    2642
A2     2636
B      2633
AC     2619
BBB    2619
BD     2617
Name: turbine_status_impute_mode, dtype: int64

## Ordinal-encoding

thay thế các nhãn bằng một số thứ tự nếu thứ tự có ý nghĩa

In [48]:
import category_encoders as ce
from sklearn.model_selection import train_test_split

In [56]:
df = df.drop(['tracking_id', 'datetime'], axis=1)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(df, df['windmill_generated_power(kW/h)'],test_size=0.2, random_state=0)

In [61]:
ord_enc = ce.OrdinalEncoder(cols=['cloud_level_impute_mode', 'turbine_status_impute_mode']).fit(X_train,y_train)

C:\Users\Ut Luom\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [62]:
df = ord_enc.transform(df)

In [63]:
df

,turbine_status_impute_mode,cloud_level_impute_mode,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),...,area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h),train
0,2,2,94.820023,-99.000000,41.723019,-0.903423,82.410573,42.523015,2563.124522,76.665560,...,26.897875,20.767246,239.836388,2730.310605,42.084666,2.217542,0.314065,24.281689,6.766521,1
1,3,2,241.832734,27.764785,-99.000000,-99.000000,44.104919,46.258870,2372.384119,78.129803,...,39.801469,20.767246,337.944723,1780.207200,107.888643,4.210346,0.448494,27.262139,5.966275,1
2,1,2,95.484724,0.371184,41.855473,12.652763,42.322098,42.878552,1657.169646,67.654469,...,36.116065,45.033197,227.850294,1666.049900,-42.931459,2.719475,0.302321,27.366127,2.874342,1
3,1,1,238.819424,-99.000000,45.443914,15.115323,44.759643,47.282101,2888.134079,95.389974,...,46.020045,44.827154,492.081520,1964.502895,42.744596,4.857385,0.367140,24.287767,14.851089,1
4,6,2,10.722890,0.371184,41.981183,1.715696,-17.616459,43.469852,781.695419,37.423065,...,34.572941,-99.000000,259.274601,1177.516152,13.387289,2.248762,0.453374,27.971650,3.519074,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12081,7,2,-15.039400,18.517204,43.545409,-1.094038,44.550547,44.486619,2424.370244,75.850209,...,28.424722,42.128526,509.795489,1798.141681,-14.365696,3.286975,0.407771,20.888674,NaN,0
12082,13,1,97.594116,-99.000000,44.747916,1.750553,43.308176,42.848779,2085.627252,73.091866,...,50.615296,47.353528,345.705839,1680.245230,-7.062661,2.201328,0.367823,24.793606,NaN,0
12083,2,1,16.186805,8.109976,43.252012,-0.691090,214.904165,41.177782,788.687177,39.514258,...,20.983939,41.768478,205.569627,2343.165194,12.331790,3.728375,0.459887,5.653451,NaN,0
12084,8,2,93.254033,-99.000000,43.841789,-1.171974,41.976913,42.039796,2079.855634,71.482324,...,29.114305,82.750271,260.578999,1732.883672,26.566581,2.247676,0.394680,24.591364,NaN,0


In [65]:
df.cloud_level_impute_mode.value_counts()

1    20153
2    19926
Name: cloud_level_impute_mode, dtype: int64